In [4]:
!pip install pyspark==3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.4.0.tar.gz (310.8 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b134c70ec430034fcdb3b5b48b51b6466ae4a16e6273b2b51b8f310929ee8cb9
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


Below section runs a local API which can be accessed by visiting the link printed. After clicking the link printed, a "Hello World" will be shown and it can be tested by appending `/predict?vol_moving_avg=12345&adj_close_rolling_med=25` to the URL.

In [8]:
# References:
# https://brightersidetech.com/running-flask-apps-in-google-colab/
# https://www.geeksforgeeks.org/get-value-of-a-particular-cell-in-pyspark-dataframe/

from flask import Flask, request, jsonify
from pyspark.ml.regression import LinearRegressionModel
#import pandas as pd

from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.appName("api").getOrCreate()

from pyspark.ml.feature import VectorAssembler
required_features = ['vol_moving_avg', 'adj_close_rolling_med']
assembler = VectorAssembler(inputCols=required_features, outputCol='features')



from google.colab.output import eval_js
print("Visit below link to access the API")
print("---------------------------------------------")
print(eval_js("google.colab.kernel.proxyPort(5000)"))
print("---------------------------------------------\n")
    
app = Flask(__name__)
# Load the model
model = LinearRegressionModel.load("/content/drive/MyDrive/StockMarket/2023-05-07-lr-model")

def volume_prediction(model, data):
    #df = pd.DataFrame(data=data)

    apiData = spark.createDataFrame([data], 
    required_features)

    apiData = assembler.transform(apiData)
    apiData = apiData.select(['features'])

    prediction = model.transform(apiData)
    #print(f'Prediction: {int(prediction.collect()[0][1])}')
    return {'volume': int(prediction.collect()[0][1])}


@app.route('/')
def home():
    return "Hello World"

"""
@app.route('/test')
def test():
  return jsonify({'test': 'You can access test API'}), 200
"""
@app.route('/predict')
def get_volume():
    vol_moving_avg = request.args.get('vol_moving_avg')
    adj_close_rolling_med = request.args.get('adj_close_rolling_med')
    
    if not vol_moving_avg or not adj_close_rolling_med:
        return jsonify({'error': 'You need to supply both vol_moving_avg and adj_close_rolling_med'}), 400

    data = [float(vol_moving_avg), float(adj_close_rolling_med)]
    return jsonify({
        **volume_prediction(model, data),
    })

if __name__ == '__main__':
    app.run()

Visit below link to access the API
---------------------------------------------
https://zh23qct8pes-496ff2e9c6d22116-5000-colab.googleusercontent.com/
---------------------------------------------

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:02:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:02:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:03:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:03:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:04:46] "GET /predict?adj_close_rolling_med=25&vol_moving_avg=12345 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:04:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:05:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 06:05:46] "GET /favicon.ico HTTP/1.1" 404 -
